In [1]:
import pandas as pd
import pymysql
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import logging
from tqdm import tqdm
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error as mse
import math

## Logging ##

import os
import sys

In [2]:
total_t1 = datetime.now()

def connect_to_mysql():
    connection = pymysql.connect(host='10.65.47.80',
                            port=4589,
                            user='netsight',
                            password='Enterasys',
                            db='netsightrpt',
                            charset='utf8',
                            cursorclass=pymysql.cursors.DictCursor)
    return connection

conn=connect_to_mysql()

In [3]:
def create_prophet_m(app_name,z1):
    
    ### --- For realtime pred ---###
    
    full_df = z1.app_rsp_time.iloc[0:len(z1)]
    full_df = full_df.reset_index()
    full_df.columns = ['ds','y']
    
    #-- Realtime prediction --##
    #model 
    model_r = Prophet(yearly_seasonality=False,changepoint_prior_scale=.2)
    model_r.fit(full_df)
    future_r = model_r.make_future_dataframe(periods=24,freq='H')
    forecast_r = model_r.predict(future_r)
    forecast_r.index = forecast_r['ds']
    #forecast 
    pred_r = pd.DataFrame(forecast_r['yhat'][len(z1):(len(z1)+24)])
    pred_r=pred_r.reset_index()
    #--- completes realtime pred ---#
    
    train_end_index=len(z1.app_rsp_time)-24
    train_df=z1.app_rsp_time.iloc[0:train_end_index]
    #train_df= train_df[train_df<cutter]
    
    
    test_df=z1.app_rsp_time.iloc[train_end_index:len(z1)]
    
    
    
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()
    train_df.columns=['ds','y']
    
    #--- removing outliers in trainset  ---#
    
    q50 = train_df.y.median()
    q100 = train_df.y.quantile(1)
    q75  = train_df.y.quantile(.75)
    print(max(train_df.y))
    if((q100-q50) >= (2*q50)):
        print('ind')
        train_df.loc[train_df.y>=(2*q50),'y'] = None
        
        print(max(train_df.y))
        print((q100-q50),(1.5*q50))
    
    
    test_df.columns=['ds','y']
    #print(train_df.tail())
    #print(test_df.head())
    
    #model 
    model = Prophet(yearly_seasonality=False,changepoint_prior_scale=.2)
    model.fit(train_df)
    future = model.make_future_dataframe(periods=len(test_df),freq='H')
    forecast = model.predict(future)
    forecast.index = forecast['ds']
    #forecast 
    pred = pd.DataFrame(forecast['yhat'][train_end_index:len(z1)])
    pred=pred.reset_index()
    pred_df=pd.merge(test_df,pred,on='ds',how='left')
    pred_df.dropna(inplace=True)
    
    df=pd.DataFrame()
    
    if(len(pred_df)>0):
        
        pred_df['error_test']=pred_df.y-pred_df.yhat
    
        t_mean =train_df.y.mean()
        pred_df['train_mean']=t_mean
        pred_df['mean_diff']= abs(pred_df.train_mean-pred_df.y)
    
        mean_mse=mse(pred_df.y,pred_df.train_mean)
        mean_RMSE=math.sqrt(mean_mse)
        pred_df['mean_APE']=abs(pred_df.mean_diff*100/pred_df.y)
        mean_MAPE=pred_df.mean_APE.mean()
    
        MSE=mse(pred_df.y,pred_df.yhat)
        RMSE=math.sqrt(MSE)
        pred_df['APE']=abs(pred_df.error_test*100/pred_df.y)
        MAPE=pred_df.APE.mean()
        print("App name:",app_name)
        print("MSE  :",MSE)
        print("RMSE :",RMSE)
        print("MAPE :",MAPE)
        #print("RMSE from mean :",mean_RMSE)
        #print("MAPE from mean :",mean_MAPE)
    
        print(pred_df.head())
        q98=pred_df['APE'].quantile(0.98)
        mape_q98=pred_df['APE'][pred_df.APE<pred_df['APE'].quantile(0.98)].mean()

        df = pd.DataFrame({'length':len(z1),#'predicted_t':[forcast_lag],
                             'test_rmse':RMSE,
                             'test_mape':MAPE,
                 #'test_ape_98':q98,
                 'test_mape_98':mape_q98},
                      #'rmse_by_mean':mean_RMSE,
                      #'mape_by_mean':mean_MAPE}
                          index=[app_name])

    return(df,model,forecast,pred_df,pred_r)

full_apps = pd.read_csv('p1/full_apps.csv')
full_apps = full_apps.loc[(full_apps['count']>1400),:]
print('fetching distinct apps completed')
full_apps = full_apps.sort_values(by='count',ascending=False)
full_apps.to_csv('p1/full_apps.csv',index=False)
a_list = list(full_apps.application)


fetching distinct apps completed


In [4]:
with conn.cursor() as cursor:
            # Read a  record
            sql = "select count(time_stamp) as count, application, source, target_address from appid_datapoint where app_rsp_time>0 group by application, source, target_address "
            cursor.execute(sql)
            app_details = pd.DataFrame(cursor.fetchall())
        


KeyboardInterrupt



In [8]:
app_details = pd.read_csv('app_details.csv')

In [9]:
app_details.shape

(1260646, 4)

In [7]:
len(app_details.loc[(app_details['count']>=1400),:])

4265

In [8]:
app_details.loc[(app_details['count']>=1400),:]

,application,count,source,target_address
105,00e0db44b70c,1874,10.6.1.101,134.141.122.74/
106,00e0db44b71e,1684,10.6.1.101,134.141.43.237/
108,00e0db44b7e7,1869,10.6.1.101,134.141.98.26/confsalemharvard
114,00e0db44b844,1874,10.6.1.101,134.141.122.56/
120,00e0db4547b5,1783,10.6.1.101,134.141.42.236/
145,00e0db46f212,1861,10.6.1.101,134.141.98.23/confsalemliberty
153,00e0db46f471,1868,10.6.1.101,134.141.98.25/confsalemsunapee
2906,10.58.36.43,1478,134.141.5.104,10.58.36.43/
3555,10.58.36.46,1470,134.141.5.104,10.58.36.46/
5069,10.6.25.196,1870,10.6.1.101,10.6.25.202/


In [9]:
len(app_details.loc[(app_details['count']>=1400),:].target_address.unique())

1981

In [10]:
len(app_details.loc[(app_details['count']>=1400),:].application.unique())

148

In [11]:
t = pd.Series(app_details.loc[(app_details['count']>=1400),:].target_address.unique())

In [12]:
targets = t.str.cat(sep="' , '")
targets

"134.141.122.74/' , '134.141.43.237/' , '134.141.98.26/confsalemharvard' , '134.141.122.56/' , '134.141.42.236/' , '134.141.98.23/confsalemliberty' , '134.141.98.25/confsalemsunapee' , '10.58.36.43/' , '10.58.36.46/' , '10.6.25.202/' , '134.141.16.125/' , '10.6.25.200/' , '134.141.16.126/' , '10.6.64.25/' , '134.141.77.24/' , '10.6.16.140/' , '10.6.25.196/' , '10.6.25.199/' , '134.141.4.23/' , '134.141.4.32/' , '134.141.77.111/' , '134.141.77.26/' , '134.141.77.27/' , '134.141.90.165/salemcafe' , '134.141.90.80/' , '134.141.90.23/' , '134.141.98.27/smays-pc1' , '134.141.98.24/confsalemcanobie' , '134.141.98.28/freid-pc' , '134.141.98.30/jeff-pc' , '134.141.98.51/confsalemmascomatrio' , '134.141.98.52/polycom_0004f2fe2f4f' , '134.141.90.50/hu1006100388.global.avaya.com' , '134.141.97.140/hu1005150089.global.avaya.com' , '134.141.97.151/hu10061002074.global.avaya.com' , '10.6.24.20/' , '134.141.77.72/' , '10.6.16.184/' , '10.127.13.60/' , '10.127.3.232/' , '10.6.16.81/' , '10.50.3.61/' ,

In [19]:
app_details.to_csv('app_details.csv',index = False)

In [15]:
len(appid_datapoint_10days)

5622539

In [10]:
a_list = app_details.loc[(app_details['count']>=1400),:].application.value_counts().index

In [10]:
targets = pd.Series(app_details.loc[(app_details['count']>=1400),:].target_address.unique())

In [11]:
len(tt)

1486

In [ ]:
comb_df = pd.DataFrame()
for a in tqdm(a_list[0:1]):
    
    qt1 = datetime.now()
    print('start quering')
    conn.close()
    conn = connect_to_mysql()
    with conn.cursor() as cursor:
            # Read a  record
            sql = "select * from appid_datapoint where application in ( '"+a+"' ) and app_rsp_time>0 and target_address in ( '"+targets+"' )"
            cursor.execute(sql)
            appid_datapoint_10days = pd.DataFrame(cursor.fetchall())
        
    if(len(appid_datapoint_10days)<1400):
        continue
    
    #appid_datapoint_10days.to_csv("p1/appid_datapoint_"+a+".csv",index=False)
    #appid_datapoint_10days = pd.read_csv("p1/appid_datapoint_"+a+".csv")

    qt2 = datetime.now()
    query_time = str(qt2-qt1)
    print('quering for '+a+' is successfull')
    print('runtime for quering for '+a+' is ',query_time)

    

    # data cleaning
    appid_datapoint_10days = appid_datapoint_10days.sort_values(by='app_rsp_time')

    dates = appid_datapoint_10days.time_stamp.astype(int)
    dates_outlook =pd.to_datetime(pd.Series(dates),unit='ms')

    appid_datapoint_10days.index = dates_outlook
    appid_datapoint_10days = appid_datapoint_10days.sort_values(by='time_stamp')
    del dates_outlook

    # run for all combinations


    prophet_df = pd.DataFrame()
    prophet_analysis_df = pd.DataFrame()
    prophet_future_df = pd.DataFrame()
    #temp = appid_datapoint_10days.copy()
   
    tt = appid_datapoint_10days.target_address.value_counts()
    tt = tt[tt>1400]
    t_list = tt.index
    s_list = appid_datapoint_10days.source.unique()
    del tt
    for i in tqdm(range(0,len(t_list))):
        
        t = t_list[i]
        #temp1 = appid_datapoint_10days[temp.target_address==t]
        for s in s_list:
            
            temp2 = len(appid_datapoint_10days[(appid_datapoint_10days.source==s) & (appid_datapoint_10days.target_address==t)])
            if(temp2<1400):
                continue
            t_df = pd.DataFrame({'application':a , 'source':s, 'target':t, 'size':temp2},index=[i])
        
            comb_df = comb_df.append(t_df)  
            
        
    qt3 = datetime.now()
    model_time  = str(qt3-qt2)


  0%|          | 0/1 [00:00<?, ?it/s]

start quering


KeyboardInterrupt: 

In [20]:
a

'DNS'

In [ ]:
comb_df = pd.DataFrame()
from datetime import datetime
q1 = datetime.now()
for a in tqdm(a_list[0:1]):
    
    qt1 = datetime.now()
    print('start quering')
    conn.close()
    conn = connect_to_mysql()
    with conn.cursor() as cursor:
            # Read a  record
            sql = "select * from appid_datapoint where application in ( '"+a+"' ) and app_rsp_time>0 and target_address in ( '"+targets+"' )"
            cursor.execute(sql)
            appid_datapoint_10days = pd.DataFrame(cursor.fetchall())
        
    if(len(appid_datapoint_10days)<1400):
        continue
    
    #appid_datapoint_10days.to_csv("p1/appid_datapoint_"+a+".csv",index=False)
    #appid_datapoint_10days = pd.read_csv("p1/appid_datapoint_"+a+".csv")

    qt2 = datetime.now()
    query_time = str(qt2-qt1)
    print('quering for '+a+' is successfull')

    logging.info(datetime.now())
    logging.info('-I- Fetching query for '+a+' is successfully...')

    # data cleaning
    appid_datapoint_10days = appid_datapoint_10days.sort_values(by='app_rsp_time')

    dates = appid_datapoint_10days.time_stamp.astype(int)
    dates_outlook =pd.to_datetime(pd.Series(dates),unit='ms')

    appid_datapoint_10days.index = dates_outlook
    appid_datapoint_10days = appid_datapoint_10days.sort_values(by='time_stamp')


    # run for all combinations


    prophet_df = pd.DataFrame()
    prophet_analysis_df = pd.DataFrame()
    prophet_future_df = pd.DataFrame()
    temp = appid_datapoint_10days.copy()
    t_list = appid_datapoint_10days.target_address.value_counts().index
    s_list = appid_datapoint_10days.source.unique()
    
    for i in tqdm(range(0,len(t_list))):
        
        t = t_list[i]
        #temp1 = temp[temp.target_address==t]
        for s in s_list:
            
            temp2 = appid_datapoint_10days[(appid_datapoint_10days.source==s) & (appid_datapoint_10days.target_address==t)]
            if(len(temp2)<=1400):
                
                continue
             
            ew_prophet,ew_model,ew_forcast,ew_pred_df,pred_r =(create_prophet_m('skype',temp2))
            ew_pred_df['application'] = a
            ew_pred_df['target'] = t
            ew_pred_df['source'] = s
            
            pred_r['application'] = a
            pred_r['target'] = t
            pred_r['source'] = s
            
            ew_prophet['target'] = t
            ew_prophet['source'] = s
            ew_prophet['application'] = a
            
            prophet_df = prophet_df.append(ew_pred_df)  
            prophet_analysis_df = prophet_analysis_df.append(ew_prophet)
            prophet_future_df = prophet_future_df.append(pred_r)
        
qt2 = datetime.now()
model_time  = str(qt2-qt1)
model_time



  0%|          | 0/1 [00:00<?, ?it/s]

start quering



Exception in thread Thread-6:
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/root/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/root/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



quering for DNS is successfull


INFO:root:2018-06-14 15:11:30.511321
INFO:root:-I- Fetching query for DNS is successfully...

  0%|          | 0/1675 [00:00<?, ?it/s]

...

545477
ind
55066.0
511704.0 50659.5
App name: skype
MSE  : 2461224.6232040445
RMSE : 1568.8290611803584
MAPE : 3.3847710965292035
                   ds      y          yhat   error_test    train_mean  \
0 2018-02-08 15:00:00  31386  32042.019156  -656.019156  33832.713734   
1 2018-02-08 16:00:00  31227  32123.476042  -896.476042  33832.713734   
2 2018-02-08 17:00:00  36521  32249.413790  4271.586210  33832.713734   
3 2018-02-08 18:00:00  30825  32407.973379 -1582.973379  33832.713734   
4 2018-02-08 19:00:00  31273  32594.088502 -1321.088502  33832.713734   

     mean_diff  mean_APE        APE  
0  2446.713734  7.795558   2.090165  
1  2605.713734  8.344425   2.870836  
2  2688.286266  7.360933  11.696247  
3  3007.713734  9.757384   5.135356  
4  2559.713734  8.185060   4.224374  
233113
ind
nan
232299.0 1221.0
App name: skype
MSE  : 288563.5100726951
RMSE : 537.1810775452678
MAPE : 32.5708025471708
                   ds     y        yhat  error_test  train_mean   mean_diff  \
0 2


  1%|          | 16/1675 [10:43<18:32:14, 40.23s/it]

138248
App name: skype
MSE  : 1908207298.8730648
RMSE : 43683.03216207713
MAPE : 768.4383283395737
                   ds      y          yhat    error_test    train_mean  \
0 2018-02-08 15:00:00  60729  80431.768741 -19702.768741  58740.763379   
1 2018-02-08 16:00:00  61217  79626.835863 -18409.835863  58740.763379   
2 2018-02-08 17:00:00  60417  79084.435515 -18667.435515  58740.763379   
3 2018-02-08 18:00:00  60999  79276.208727 -18277.208727  58740.763379   
4 2018-02-08 19:00:00  61448  79197.588997 -17749.588997  58740.763379   

     mean_diff  mean_APE        APE  
0  1988.236621  3.273949  32.443756  
1  2476.236621  4.045015  30.073078  
2  1676.236621  2.774445  30.897654  
3  2258.236621  3.702088  29.963128  
4  2707.236621  4.405736  28.885544  
4911570
ind
29820.0
4896591.5 22467.75
App name: skype
MSE  : 1492931983490.0332
RMSE : 1221855.9585687805
MAPE : 37.85797051687292
                   ds      y          yhat   error_test    train_mean  \
0 2018-02-08 19:00:00  


  1%|          | 17/1675 [11:14<18:16:10, 39.67s/it]

526273
ind
82013.0
484919.5 62030.25
App name: skype
MSE  : 76101669.5104394
RMSE : 8723.627084558315
MAPE : 13.1747663375118
                   ds      y          yhat   error_test    train_mean  \
0 2018-02-08 15:00:00  44886  50855.972218 -5969.972218  41029.894565   
1 2018-02-08 16:00:00  48536  49393.820217  -857.820217  41029.894565   
2 2018-02-08 17:00:00  48182  46106.547962  2075.452038  41029.894565   
3 2018-02-08 18:00:00  51160  42259.147885  8900.852115  41029.894565   
4 2018-02-08 19:00:00  42467  39296.866785  3170.133215  41029.894565   

      mean_diff   mean_APE        APE  
0   3856.105435   8.590887  13.300299  
1   7506.105435  15.465027   1.767390  
2   7152.105435  14.843936   4.307526  
3  10130.105435  19.800832  17.398069  
4   1437.105435   3.384052   7.464933  


In [20]:
targets1 = targets.str.cat(sep="' , '")

In [21]:
    conn = connect_to_mysql()
    with conn.cursor() as cursor:
            # Read a  record
            sql = "select * from appid_datapoint where application in ( '"+a+"' ) and app_rsp_time>0 and target_address in ( '"+targets1+"' )"
            cursor.execute(sql)
            appid_datapoint_10days = pd.DataFrame(cursor.fetchall())
        
    

KeyboardInterrupt: 

In [16]:
print('hi')

hi


##  Parallel processing

In [1]:
from pyspark.sql import SQLContext


from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import os
import pandas as pd

#os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[2] pyspark-shell"
#os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-oracle"

#sc = SparkContext()
#spark = SparkSession(sc)

#sqlContext = SQLContext(sc)


In [2]:
from pyspark.sql import SparkSession
import datetime
# initialise sparkContext
spark1 = SparkSession.builder \
    .master('local') \
    .appName('p7_sample') \
    .config('spark.executor.memory', '24gb') \
    .config("spark.cores.max", "16") \
    .getOrCreate()

sc = spark1.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
# to read parquet file
df = sqlContext.read.parquet('appid_datapoint_parquet1')

In [4]:
import multiprocessing as mp
from joblib import Parallel, delayed
a = 'Salesforce'
df1 = df[df.application==a]
df1.head()

Row(time_stamp=1510801200000, source='134.141.5.104', zone='\\N', target_address='10.51.1.95/', application='Salesforce', applicationgroup='Business Applications', target_attribute_id=215904025, byte_count=0, tx_byte_count=0, rx_byte_count=0, flow_count=0, tx_flow_count=0, rx_flow_count=0, tcp_rsp_time=0, app_rsp_time=0, user_data='\\N')

In [5]:
t1 = datetime.datetime.now()
appid_datapoint_10days = df1
t2 = datetime.datetime.now()
str(t2-t1)
appid_datapoint_10days.head()

Row(time_stamp=1510801200000, source='134.141.5.104', zone='\\N', target_address='10.51.1.95/', application='Salesforce', applicationgroup='Business Applications', target_attribute_id=215904025, byte_count=0, tx_byte_count=0, rx_byte_count=0, flow_count=0, tx_flow_count=0, rx_flow_count=0, tcp_rsp_time=0, app_rsp_time=0, user_data='\\N')

In [6]:
app_details = pd.read_csv('app_details.csv')
targets = pd.Series(app_details.loc[(app_details['count']>=1400),:].target_address.unique())
s_list = app_details.source.unique()
s = s_list[0]
t = targets[1]

In [7]:
from tqdm import tqdm
temp2 = appid_datapoint_10days[(appid_datapoint_10days.source==s) & (appid_datapoint_10days.target_address==t)]
t1 = datetime.datetime.now()

ts = temp2.select('time_stamp').collect()
ts_array = [int(i.time_stamp) for i in ts]
y = temp2.select('app_rsp_time').collect()
y_array = [int(i.app_rsp_time) for i in y]

t2 = datetime.datetime.now()
print((str(t2-t1)))
dates_outlook = pd.to_datetime(pd.Series(ts_array),unit='ms')
df2 = pd.DataFrame({'ds':ts_array,'app_rsp_time':y_array})
df2.index = dates_outlook  
df2.head()

0:00:22.871352


,app_rsp_time,ds
2017-11-16 03:00:00,0,1510801200000
2017-11-16 04:00:00,0,1510804800000
2017-11-16 05:00:00,0,1510808400000
2017-11-16 06:00:00,0,1510812000000
2017-11-16 07:00:00,0,1510815600000


In [8]:
df2.tail()

,app_rsp_time,ds
2018-02-09 14:00:00,0,1518184800000
2018-01-29 04:00:00,0,1517198400000
2018-01-29 05:00:00,0,1517202000000
2018-01-29 06:00:00,0,1517205600000
2018-01-29 07:00:00,0,1517209200000


In [27]:

def forapp(t):
    
    q1 = datetime.datetime.now()
    #for a in tqdm(a_list[0:1]):
    
    qt1 = datetime.datetime.now()
    print('start quering')
    #conn.close()
    
    qt2 = datetime.datetime.now()
    query_time = str(qt2-qt1)
    print('quering for '+a+' is successfull')

    
    # run for all combinations


    prophet_df = pd.DataFrame()
    prophet_analysis_df = pd.DataFrame()
    prophet_future_df = pd.DataFrame()
   
    
    #for i in tqdm(range(0,len(t_list))):
        
    #t = t_list[i]
        #temp1 = temp[temp.target_address==t]
    for s in tqdm(s_list):
    #if(s):
            
        temp2 = appid_datapoint_10days[(appid_datapoint_10days.source==s) & (appid_datapoint_10days.target_address==t)]
        
        
        t1 = datetime.datetime.now()

        ts = temp2.select('time_stamp').collect()
        ts_array = [int(i.time_stamp) for i in ts]
        y = temp2.select('app_rsp_time').collect()
        y_array = [int(i.app_rsp_time) for i in y]

        t2 = datetime.datetime.now()
        print((str(t2-t1)))
        dates_outlook = pd.to_datetime(pd.Series(ts_array),unit='ms')
        
        df2 = pd.DataFrame({'ds':ts_array,'app_rsp_time':y_array})
        df2.index = dates_outlook
        df2 = df2[df2.app_rsp_time!=0]
        #df2 = df2.dropna()
        if(len(df2)>1400):
            
        
            ew_prophet,ew_model,ew_forcast,ew_pred_df,pred_r =(create_prophet_m(a,df2))
            ew_pred_df['application'] = a
            ew_pred_df['target'] = t
            ew_pred_df['source'] = s
            
            pred_r['application'] = a
            pred_r['target'] = t
            pred_r['source'] = s
            
            ew_prophet['target'] = t
            ew_prophet['source'] = s
            ew_prophet['application'] = a
            
            prophet_df = prophet_df.append(ew_pred_df)  
            prophet_analysis_df = prophet_analysis_df.append(ew_prophet)
            prophet_future_df = prophet_future_df.append(pred_r)
        
    qt2 = datetime.datetime.now()
    print(str(qt2-qt1))
    
    return prophet_df, prophet_analysis_df, prophet_future_df

In [32]:
a1,a2,a3=forapp(t)

  0%|          | 0/7 [00:00<?, ?it/s]

start quering
quering for Salesforce is successfull


 14%|█▍        | 1/7 [00:22<02:14, 22.34s/it]

0:00:22.284866


 29%|██▊       | 2/7 [00:43<01:49, 21.96s/it]

...

0:00:20.954944


 86%|████████▌ | 6/7 [02:09<00:21, 21.51s/it]

0:00:21.536157


100%|██████████| 7/7 [02:30<00:00, 21.50s/it]

0:00:21.412236
0:02:30.487154


In [29]:
t1 = datetime.datetime.now()
targets2 = targets[0:7]
pool = Parallel(n_jobs=-1,verbose=5,pre_dispatch='all')
results = pool(delayed(forapp)(t) for t in targets2) #doc_list[0:50000]
t2 = datetime.datetime.now()
print((str(t2-t1)))

start quering
quering for Salesforce is successfull
start quering
quering for Salesforce is successfull


  0%|          | 0/7 [00:00<?, ?it/s]

start quering
start quering
quering for Salesforce is successfull
quering for Salesforce is successfull


  0%|          | 0/7 [00:00<?, ?it/s]

start quering
start quering
quering for Salesforce is successfull
start quering


...

  0%|          | 0/7 [00:00<?, ?it/s]

quering for Salesforce is successfull


  0%|          | 0/7 [00:00<?, ?it/s]Process ForkPoolWorker-77:
Process ForkPoolWorker-72:
Process ForkPoolWorker-76:
Process ForkPoolWorker-70:
Process ForkPoolWorker-78:
Process ForkPoolWorker-73:
Process ForkPoolWorker-75:
Process ForkPoolWorker-74:
Process ForkPoolWorker-79:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/root/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/root/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/root/anaconda3/lib/python3.6/multiprocessi

KeyboardInterrupt
KeyboardInterrupt
  File "/root/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
  File "/root/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/root/anaconda3/lib/python3.6/site-packages/joblib/pool.py", line 360, in get
    racquire()
KeyboardInterrupt


KeyboardInterrupt: 

In [26]:
len(results)

30